In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
# from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

# ds_train = ds_xy.take(20000)
# ds_test = ds_xy.skip(20000).take(10000)

(220025, 198022)

In [8]:
data_aug = Sequential([
    RandomFlip("horizontal_and_vertical"),
    # RandomZoom(0.2, 0.2)
])

In [9]:
model = Sequential()

model.add(data_aug)

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [10]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
3095/3095 - 155s - loss: 0.4877 - accuracy: 0.8235 - val_loss: 1.9654 - val_accuracy: 0.6393
Epoch 2/8
3095/3095 - 153s - loss: 0.3421 - accuracy: 0.8775 - val_loss: 0.4554 - val_accuracy: 0.8413
Epoch 3/8
3095/3095 - 153s - loss: 0.2817 - accuracy: 0.9041 - val_loss: 0.6435 - val_accuracy: 0.7691
Epoch 4/8
3095/3095 - 154s - loss: 0.2538 - accuracy: 0.9181 - val_loss: 0.3852 - val_accuracy: 0.8756
Epoch 5/8
3095/3095 - 154s - loss: 0.2374 - accuracy: 0.9249 - val_loss: 0.6943 - val_accuracy: 0.7342
Epoch 6/8
3095/3095 - 154s - loss: 0.2311 - accuracy: 0.9277 - val_loss: 0.2522 - val_accuracy: 0.9157
Epoch 7/8
3095/3095 - 154s - loss: 0.2248 - accuracy: 0.9307 - val_loss: 0.2930 - val_accuracy: 0.8967
Epoch 8/8
3095/3095 - 154s - loss: 0.2221 - accuracy: 0.9322 - val_loss: 0.3392 - val_accuracy: 0.8715


In [11]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
1548/1548 - 154s - loss: 0.1793 - accuracy: 0.9485 - val_loss: 0.1807 - val_accuracy: 0.9472
Epoch 2/8
1548/1548 - 152s - loss: 0.1695 - accuracy: 0.9520 - val_loss: 0.1641 - val_accuracy: 0.9521
Epoch 3/8
1548/1548 - 152s - loss: 0.1638 - accuracy: 0.9529 - val_loss: 0.1536 - val_accuracy: 0.9550
Epoch 4/8
1548/1548 - 150s - loss: 0.1609 - accuracy: 0.9535 - val_loss: 0.1440 - val_accuracy: 0.9598
Epoch 5/8
1548/1548 - 163s - loss: 0.1562 - accuracy: 0.9549 - val_loss: 0.1370 - val_accuracy: 0.9624
Epoch 6/8
1548/1548 - 162s - loss: 0.1539 - accuracy: 0.9554 - val_loss: 0.1448 - val_accuracy: 0.9576
Epoch 7/8
1548/1548 - 159s - loss: 0.1515 - accuracy: 0.9564 - val_loss: 0.1451 - val_accuracy: 0.9583
Epoch 8/8
1548/1548 - 151s - loss: 0.1500 - accuracy: 0.9563 - val_loss: 0.1762 - val_accuracy: 0.9518


In [12]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
774/774 - 157s - loss: 0.1371 - accuracy: 0.9611 - val_loss: 0.1286 - val_accuracy: 0.9656
Epoch 2/8
774/774 - 163s - loss: 0.1345 - accuracy: 0.9617 - val_loss: 0.1320 - val_accuracy: 0.9633
Epoch 3/8
774/774 - 152s - loss: 0.1327 - accuracy: 0.9627 - val_loss: 0.1294 - val_accuracy: 0.9641
Epoch 4/8
774/774 - 153s - loss: 0.1330 - accuracy: 0.9625 - val_loss: 0.1315 - val_accuracy: 0.9638
Epoch 5/8
774/774 - 161s - loss: 0.1312 - accuracy: 0.9630 - val_loss: 0.1340 - val_accuracy: 0.9617
Epoch 6/8
774/774 - 152s - loss: 0.1307 - accuracy: 0.9634 - val_loss: 0.1411 - val_accuracy: 0.9584
Epoch 7/8
774/774 - 161s - loss: 0.1299 - accuracy: 0.9631 - val_loss: 0.1348 - val_accuracy: 0.9606
Epoch 8/8
774/774 - 152s - loss: 0.1287 - accuracy: 0.9639 - val_loss: 0.1303 - val_accuracy: 0.9629


In [13]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(64).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 162s - loss: 0.1471 - accuracy: 0.9564 - val_loss: 0.1320 - val_accuracy: 0.9625
Epoch 2/2
3095/3095 - 152s - loss: 0.1472 - accuracy: 0.9564 - val_loss: 0.1502 - val_accuracy: 0.9551


In [14]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
774/774 - 155s - loss: 0.1283 - accuracy: 0.9634 - val_loss: 0.1242 - val_accuracy: 0.9646
Epoch 2/5
774/774 - 151s - loss: 0.1262 - accuracy: 0.9642 - val_loss: 0.1266 - val_accuracy: 0.9644
Epoch 3/5
774/774 - 151s - loss: 0.1248 - accuracy: 0.9652 - val_loss: 0.1289 - val_accuracy: 0.9626
Epoch 4/5
774/774 - 151s - loss: 0.1247 - accuracy: 0.9648 - val_loss: 0.1326 - val_accuracy: 0.9608
Epoch 5/5
774/774 - 151s - loss: 0.1237 - accuracy: 0.9649 - val_loss: 0.1439 - val_accuracy: 0.9559


In [15]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(512).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(512).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
387/387 - 162s - loss: 0.1339 - accuracy: 0.9613 - val_loss: 0.3068 - val_accuracy: 0.9090
Epoch 2/2
387/387 - 152s - loss: 0.1322 - accuracy: 0.9619 - val_loss: 0.1906 - val_accuracy: 0.9389


In [16]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 154s - loss: 0.1222 - accuracy: 0.9654 - val_loss: 0.1233 - val_accuracy: 0.9647
Epoch 2/10
387/387 - 152s - loss: 0.1201 - accuracy: 0.9662 - val_loss: 0.1325 - val_accuracy: 0.9605
Epoch 3/10
387/387 - 152s - loss: 0.1192 - accuracy: 0.9664 - val_loss: 0.1181 - val_accuracy: 0.9663
Epoch 4/10
387/387 - 152s - loss: 0.1179 - accuracy: 0.9671 - val_loss: 0.1271 - val_accuracy: 0.9626
Epoch 5/10
387/387 - 152s - loss: 0.1172 - accuracy: 0.9673 - val_loss: 0.1316 - val_accuracy: 0.9605
Epoch 6/10
387/387 - 152s - loss: 0.1170 - accuracy: 0.9677 - val_loss: 0.1326 - val_accuracy: 0.9603
Epoch 7/10
387/387 - 152s - loss: 0.1166 - accuracy: 0.9673 - val_loss: 0.1253 - val_accuracy: 0.9630
Epoch 8/10
387/387 - 152s - loss: 0.1148 - accuracy: 0.9678 - val_loss: 0.1328 - val_accuracy: 0.9604
Epoch 9/10
387/387 - 155s - loss: 0.1150 - accuracy: 0.9682 - val_loss: 0.1250 - val_accuracy: 0.9633
Epoch 10/10
387/387 - 155s - loss: 0.1143 - accuracy: 0.9684 - val_loss: 0.1343 - 

In [17]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 166s - loss: 0.1137 - accuracy: 0.9684 - val_loss: 0.1215 - val_accuracy: 0.9646
Epoch 2/10
387/387 - 160s - loss: 0.1129 - accuracy: 0.9686 - val_loss: 0.1200 - val_accuracy: 0.9655
Epoch 3/10
387/387 - 162s - loss: 0.1125 - accuracy: 0.9689 - val_loss: 0.1183 - val_accuracy: 0.9660
Epoch 4/10
387/387 - 158s - loss: 0.1125 - accuracy: 0.9687 - val_loss: 0.1211 - val_accuracy: 0.9651
Epoch 5/10
387/387 - 165s - loss: 0.1122 - accuracy: 0.9691 - val_loss: 0.1217 - val_accuracy: 0.9649
Epoch 6/10
387/387 - 162s - loss: 0.1119 - accuracy: 0.9688 - val_loss: 0.1204 - val_accuracy: 0.9654
Epoch 7/10
387/387 - 163s - loss: 0.1121 - accuracy: 0.9688 - val_loss: 0.1189 - val_accuracy: 0.9659
Epoch 8/10
387/387 - 165s - loss: 0.1109 - accuracy: 0.9689 - val_loss: 0.1202 - val_accuracy: 0.9653
Epoch 9/10
387/387 - 165s - loss: 0.1114 - accuracy: 0.9690 - val_loss: 0.1196 - val_accuracy: 0.9654
Epoch 10/10
387/387 - 164s - loss: 0.1120 - accuracy: 0.9688 - val_loss: 0.1188 - 

In [18]:
history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 163s - loss: 0.1110 - accuracy: 0.9691 - val_loss: 0.1163 - val_accuracy: 0.9670
Epoch 2/10
387/387 - 160s - loss: 0.1108 - accuracy: 0.9692 - val_loss: 0.1220 - val_accuracy: 0.9643
Epoch 3/10
387/387 - 166s - loss: 0.1115 - accuracy: 0.9694 - val_loss: 0.1186 - val_accuracy: 0.9659
Epoch 4/10
387/387 - 166s - loss: 0.1113 - accuracy: 0.9691 - val_loss: 0.1164 - val_accuracy: 0.9670
Epoch 5/10
387/387 - 161s - loss: 0.1109 - accuracy: 0.9690 - val_loss: 0.1179 - val_accuracy: 0.9666
Epoch 6/10
387/387 - 165s - loss: 0.1107 - accuracy: 0.9693 - val_loss: 0.1201 - val_accuracy: 0.9651
Epoch 7/10
387/387 - 163s - loss: 0.1113 - accuracy: 0.9688 - val_loss: 0.1226 - val_accuracy: 0.9640
Epoch 8/10
387/387 - 163s - loss: 0.1110 - accuracy: 0.9692 - val_loss: 0.1194 - val_accuracy: 0.9654
Epoch 9/10
387/387 - 164s - loss: 0.1106 - accuracy: 0.9693 - val_loss: 0.1226 - val_accuracy: 0.9639
Epoch 10/10
387/387 - 159s - loss: 0.1104 - accuracy: 0.9691 - val_loss: 0.1188 - 